In [46]:
import numpy as np
import math
from math import e
import pandas as pd

In [27]:
# define Pauli matrices
I = np.matrix([[1,0],[0,1]])
X = np.matrix([[0,1],[1,0]])
Y = np.matrix([[0,0-1j],[0+1j,0]])
Z = np.matrix([[1,0],[0,-1]])

In [28]:
# any linear combination of I,X,Y,Z
def generate_combination(i,x,y,z):
    return i*I+x*X+y*Y+z*Z

In [29]:
# function for checking whether or not matrix is unitary (all valid hamiltonians are unitary)
def check_unitary(A):
    Adag = A.getH()
    if not (np.array_equal(np.matmul(Adag,A),I) and np.array_equal(np.matmul(A,Adag),I)):
        return False
    return True

In [30]:
def generate_hamiltonian(alpha,beta,gamma,delta):
    term1 = e**((alpha-beta/2-delta/2)*1j)*math.cos(gamma/2)
    term2 = e**((alpha-beta/2+delta/2)*1j)*math.sin(gamma/2)*(-1)
    term3 = e**((alpha+beta/2-delta/2)*1j)*math.sin(gamma/2)
    term4 = e**((alpha+beta/2+delta/2)*1j)*math.cos(gamma/2)
    hamiltonian = np.matrix([[term1,term2],[term3,term4]])
    # we verify that the hamiltonian is unitary, and if it isn't we return the identity
    if check_unitary(hamiltonian):
        return hamiltonian
    else:
        return I

#below are other possible functions to generate unitary matrices, but they do not have as good success rates
'''
def generate_hamiltonian(alpha,beta,xi,zeta):
    first_mat = np.matrix([[math.cos(alpha),-math.sin(alpha)],[math.sin(alpha),math.cos(alpha)]])
    second_mat = np.matrix([[e**((xi)*1j),0],[0,e**((zeta)*1j)]])
    third_mat = np.matrix([[math.cos(beta),math.sin(beta)],[-math.sin(beta),math.cos(beta)]])
    return np.matmul(np.matmul(first_mat,second_mat),third_mat)

def generate_hamiltonian(x,y,z):
    return np.matmul(np.matmul(X**x,Y**y),Z**z)
'''

'\ndef generate_hamiltonian(alpha,beta,xi,zeta):\n    first_mat = np.matrix([[math.cos(alpha),-math.sin(alpha)],[math.sin(alpha),math.cos(alpha)]])\n    second_mat = np.matrix([[e**((xi)*1j),0],[0,e**((zeta)*1j)]])\n    third_mat = np.matrix([[math.cos(beta),math.sin(beta)],[-math.sin(beta),math.cos(beta)]])\n    return np.matmul(np.matmul(first_mat,second_mat),third_mat)\n\ndef generate_hamiltonian(x,y,z):\n    return np.matmul(np.matmul(X**x,Y**y),Z**z)\n'

In [31]:
# change values of alpha, beta, gamma, and delta to see what sort of unitary matrices we return
a=1
b=1
c=1
d=1
A = generate_hamiltonian(a,b,c,d)

In [32]:
# should always be true
check_unitary(A)

True

In [33]:
# if A is identity, maybe try different values of alpha, beta, gamma, and delta
A

matrix([[ 0.87758256+0.j        , -0.25903472-0.40342268j],
        [ 0.25903472+0.40342268j, -0.36520321+0.79798357j]])

In [34]:
# generate random 2 dimensional vector
vec = np.random.rand(2)

In [35]:
def get_magnitude(v):
    return math.sqrt(v[0]**2+v[1]**2)

In [36]:
# change vector to have norm of 1
vec = vec/get_magnitude(vec)

In [39]:
def evolve(v,A):
    return np.matmul(A,v).A1

In [55]:
# v is the inital state as a complex vector with two dimensions
# A is the hamiltonian defining the evolution of the state
# n is the number of evolutions, the number of state vectors in the returned evolution_matrix
def generate_evolution_matrix(v,A,n):
    evolution_matrix = np.array([v,evolve(v,A)])
    v = evolve(evolve(v,A),A)
    for i in range(0,n-2):
        evolution_matrix = np.vstack( (evolution_matrix,np.array([v])) )
        v = evolve(v,A)
    return evolution_matrix

In [56]:
evolution_matrix = generate_evolution_matrix(vec,A,1000)

In [57]:
split_evolution_matrix = np.zeros((evolution_matrix.shape[0],4))

In [58]:
#extract real and imaginary parts out of evolution matrix
for row in range(0,evolution_matrix.shape[0]):
    split_evolution_matrix[row, 0] = np.real(evolution_matrix[row, 0])
    split_evolution_matrix[row, 1] = np.imag(evolution_matrix[row, 0])
    split_evolution_matrix[row, 2] = np.real(evolution_matrix[row, 1])
    split_evolution_matrix[row, 3] = np.imag(evolution_matrix[row, 1])

In [59]:
# export evolution matrix as csv
mat_df = pd.DataFrame(split_evolution_matrix)
mat_df.to_csv("evolution_data.csv")